In [ ]:
#Importar librerias
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt 

In [ ]:
#leer el dataframe 
dataframe=pd.read_csv('//content/95-mero-343.csv')
dataframe = dataframe.drop(['id', 'aislamiento', 'MEROI'], axis='columns')
#defino las variables "X" y "y"
X = dataframe.drop(['MEROPENEM'], axis='columns')
y = dataframe.iloc[:, -1]  
ylog=np.log2(y)

In [ ]:
#perform permutation importance
modelo = RandomForestRegressor ()
modelo.fit(X, ylog)
characteristic = dataframe.columns
results = permutation_importance(modelo, X, ylog, scoring='neg_mean_squared_error')
# get importance
importances = results.importances_mean
characteristics_importances = [(characteristic, round(importance, 2)) for characteristic, importance in zip(characteristic, importances)]
characteristics_importances = sorted(characteristics_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in characteristics_importances];

In [ ]:
#funcion para calcular la excatitud con +/- una dilucion 
def within_dillution(y_test, y_pred1, dill=2):
    frac = np.array(y_test) / np.array(y_pred1)
    return ((frac >= 1/dill) & (frac <= dill)).mean()

In [ ]:
genes=[]
linea=[]
resultados=[]
for pair in characteristics_importances [: 53]:#[:final]
  linea=[]
  genes.append(pair[0])
  print(genes)
  X = dataframe[genes]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=27)
  #convierto la variable "y" en log 
  y_train1 = np.log2(y_train)
  y_test1 = np.log2(y_test)
  #ajuste de hiperparametros por gridesearch
  modelo1 = RandomForestRegressor()
  n_estimators = [50, 100, 200, 300]
  max_depth = [5, 6, 7, 8, 10]
  min_samples_split = [2, 3, 5, 6] 
  min_samples_leaf = [1, 2, 3] 
  max_features = ['sqrt', 10, 20, 30, 45, 57]
  hyperF = dict(max_depth = max_depth, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf)
  gridF = GridSearchCV(modelo1, hyperF, cv = 10, verbose = 1,n_jobs = -1)
  bestF = gridF.fit(X_train, y_train)
  print('Best Params: ', gridF.best_params_)
  linea.append(gridF.best_params_)
  #definir el modelo
  modelo = RandomForestRegressor(max_depth= gridF.best_params_['max_depth'], min_samples_leaf= gridF.best_params_['min_samples_leaf'], min_samples_split= gridF.best_params_['min_samples_split'])
  #entreno modelo y realiazo predicciones 
  modelo.fit(X_train, y_train1) #entreno
  y_pred = modelo.predict(X_test)
  #hago predicciones 
  #print(y_pred)
  y_pred1 = 2**np.round(modelo.predict(X_test))
  #print(y_pred1)
  verdaderos = y_test.to_numpy()
  #print(verdaderos)
  #obtengo los valores que muestran el desempeño del modelo
  maete=metrics.mean_absolute_error(y_test1, y_pred)
  print('Mean Absolute Error_test:', maete)
  linea.append(maete)
  maetr=metrics.mean_absolute_error(y_train1, modelo.predict(X_train))
  print('Mean Absolute Error_train:', maetr)
  linea.append(maetr)
  msete=metrics.mean_squared_error(y_test1, y_pred)
  print('Mean Squared Error_test:', msete)
  linea.append(msete)
  msetr=metrics.mean_squared_error(y_train1, modelo.predict(X_train))
  print('Mean Squared Error_train:', msetr)
  linea.append(msetr)
  rmsete=np.sqrt(metrics.mean_squared_error(y_test1, y_pred))
  print('Root Mean Squared Error_test:', rmsete)
  linea.append(rmsete)
  rmstr=np.sqrt(metrics.mean_squared_error(y_train1, modelo.predict(X_train)))
  print('RMSE_train:', rmstr)
  linea.append(rmstr)
  r2tr=metrics.r2_score(y_train1, modelo.predict(X_train))
  print('r2_train:', r2tr)
  linea.append(r2tr)
  r2te=metrics.r2_score(y_test1, (y_pred))
  print('r2_teste:', r2te)
  linea.append(r2te)
  #saco accuracy con +/- 1 dilucion, VME y ME
  acc2 = within_dillution(y_test, y_pred1, dill=2)
  acc4 = within_dillution(y_test, y_pred1, dill=4)
  threshold = 2
  threshold1 = 4 #colacel el punto de corte para el antibiotico que esta probando
  predDF = pd.DataFrame({'y_pred1':y_pred1, 'y_test':y_test})
  vme = len(predDF[(predDF.y_pred1<=threshold) & (predDF.y_test>threshold1)])
  me = len(predDF[(predDF.y_pred1>threshold1) & (predDF.y_test<=threshold)])
  #print(vme)
  linea.append(vme)
  #print(me)
  linea.append(me)
  #print(acc2)
  linea.append(acc2)
  #print(acc4)
  resultados.append(linea)

['Escherichia_coli_EF-Tu_mutants_conferring_resistance_to_Pulvomycin']
Fitting 10 folds for each of 60 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   50.0s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  1.1min finished


Best Params:  {'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 3}
Mean Absolute Error_test: 1.228824429396521
Mean Absolute Error_train: 1.2951670443755106
Mean Squared Error_test: 2.995656752897484
Mean Squared Error_train: 2.550151813720123
Root Mean Squared Error_test: 1.730796566005804
RMSE_train: 1.5969194762792904
r2_train: 0.44718054553713427
r2_teste: 0.34768012255100045
['Escherichia_coli_EF-Tu_mutants_conferring_resistance_to_Pulvomycin', 'OXA-23']
Fitting 10 folds for each of 60 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   27.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   55.1s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  1.2min finished


Best Params:  {'max_depth': 7, 'min_samples_leaf': 3, 'min_samples_split': 5}
Mean Absolute Error_test: 0.8678949973824482
Mean Absolute Error_train: 0.907123093292841
Mean Squared Error_test: 2.2450788961027643
Mean Squared Error_train: 1.7724621880508957
Root Mean Squared Error_test: 1.4983587341163545
RMSE_train: 1.331338494918139
r2_train: 0.6157673536992447
r2_teste: 0.5111223644188957
['Escherichia_coli_EF-Tu_mutants_conferring_resistance_to_Pulvomycin', 'OXA-23', 'OXA-65']
Fitting 10 folds for each of 60 candidates, totalling 600 fits


...

Best Params:  {'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 5}
Mean Absolute Error_test: 0.6870622965876593
Mean Absolute Error_train: 0.4692558918244707
Mean Squared Error_test: 2.004835178143184
Mean Squared Error_train: 0.7574717684854578
Root Mean Squared Error_test: 1.4159220240335215
RMSE_train: 0.870328540544005
r2_train: 0.8357959994490313
r2_teste: 0.5634366866474706
['Escherichia_coli_EF-Tu_mutants_conferring_resistance_to_Pulvomycin', 'OXA-23', 'OXA-65', 'mphE', 'Escherichia_coli_EF-Tu_mutants_conferring_resistance_to_Pulvomycin_cp1', "APH(3')-VI", "AAC(6')-Ian", 'adeS', 'Escherichia_coli_gyrA_conferring_resistance_to_fluoroquinolones', 'OXA-72', 'msrE', 'OXA-259', 'abeS', 'cmlB1', 'ADC-2', 'ADC-5', 'Salmonella_serovars_gyrB_conferring_resistance_to_fluoroquinolone', 'OXA-64', 'adeL', 'Clostridioides_difficile_gyrB_conferring_resistance_to_fluoroquinolone', 'sul2', 'tet(B)', 'ugd', 'adeC', 'OXA-146', 'adeN', 'Acinetobacter_baumannii_AbaF', 'AAC(3)-IIe', 'TEM-

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  1.4min finished


Best Params:  {'max_depth': 7, 'min_samples_leaf': 3, 'min_samples_split': 2}
Mean Absolute Error_test: 0.6852799738193714
Mean Absolute Error_train: 0.47697765314777546
Mean Squared Error_test: 2.151807832013956
Mean Squared Error_train: 0.7813195048997589
Root Mean Squared Error_test: 1.4669041659269892
RMSE_train: 0.8839227935174875
r2_train: 0.8306263101137535
r2_teste: 0.5314326249442713
['Escherichia_coli_EF-Tu_mutants_conferring_resistance_to_Pulvomycin', 'OXA-23', 'OXA-65', 'mphE', 'Escherichia_coli_EF-Tu_mutants_conferring_resistance_to_Pulvomycin_cp1', "APH(3')-VI", "AAC(6')-Ian", 'adeS', 'Escherichia_coli_gyrA_conferring_resistance_to_fluoroquinolones', 'OXA-72', 'msrE', 'OXA-259', 'abeS', 'cmlB1', 'ADC-2', 'ADC-5', 'Salmonella_serovars_gyrB_conferring_resistance_to_fluoroquinolone', 'OXA-64', 'adeL', 'Clostridioides_difficile_gyrB_conferring_resistance_to_fluoroquinolone', 'sul2', 'tet(B)', 'ugd', 'adeC', 'OXA-146', 'adeN', 'Acinetobacter_baumannii_AbaF', 'AAC(3)-IIe', 'TEM

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  1.4min finished


Best Params:  {'max_depth': 8, 'min_samples_leaf': 3, 'min_samples_split': 5}
Mean Absolute Error_test: 0.6805005277542797
Mean Absolute Error_train: 0.46952137674578637
Mean Squared Error_test: 2.0136804410455316
Mean Squared Error_train: 0.7600737578245578
Root Mean Squared Error_test: 1.4190420857203396
RMSE_train: 0.8718220906954341
r2_train: 0.8352319427057345
r2_teste: 0.561510584530837


In [ ]:
resultadosdf=pd.DataFrame(resultados)
print(resultadosdf)
resultadosdf.to_excel ('MERO-95-RF-PER.xlsx', index = None, header=True)